In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import pandas as pd
import gc

In [4]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [5]:
!ls

'BETO twitter.ipynb'		  exp-Copy1.ipynb	   results
'Muestreo y estimaciones.ipynb'   exp.ipynb		   tweets_hombres.csv
 README.md			  gobernadoras_small.csv   tweets_mujeres.csv
 SSNMX_sismos.csv		  gobernadores_small.csv
'ds_salaries 2.csv'		  my_model_checkpoints


Columnas que se preservan de mis datos: nombre, puesto, usuario, (U)ubicación, fecha d einicio, fecha de término, seguidores, tweets hechos, tweets respuesta, componente sg, edad, comentarios,

Columnas de Twint: id, conversation id, created at, date, timezone, place, tweet, language, hashtags, cashtags, user_id, user id str, username, name, day, hour, link, urls, photos, video, thmbnail, retweet, nlikes, nreplies, nretweets, quote_url, search, near, geo, source, user rt id, user rt, retweet_id, reply_to, retweet date, translate, trans src, trans dest

Columnas con las que me puedo quedar: id, tweet, language, hastags, nlikes, nreplies, nretweets.

In [8]:
mujeres = pd.read_csv("gobernadoras_small.csv",usecols=['id','tweet'])


In [9]:
hombres = pd.read_csv("gobernadores_small.csv",usecols=['id','tweet'])

Hasta aquí tenemos los datos separados en esfera de hombres y esfera de mujeres.

In [10]:
#import torch
import transformers
from transformers import BertForMaskedLM, BertTokenizer
from transformers import AlbertForMaskedLM, AlbertTokenizer
from datasets import Dataset
import datasets

In [11]:
import math

In [12]:
#import torchtext
from collections import Counter
#from torchtext.vocab import Vocab
#from torch import Tensor
import io
import time

Emparejar de manera aleatoria los datasets

In [13]:
import numpy as np

np.random.seed(10)

remove_n = len(mujeres) - len(hombres)
drop_indices = np.random.choice(mujeres.index, remove_n, replace=False)
mujeres = mujeres.drop(drop_indices)

In [14]:
mujeres.columns

Index(['id', 'tweet'], dtype='object')

In [15]:
data = {"mujeres_tweet":list(mujeres.tweet),
        "hombres_tweet":list(hombres.tweet)}
raw_dataset = pd.DataFrame(data)
raw_dataset

,mujeres_tweet,hombres_tweet
0,"@MarinadelPilar ¿estrategia?, puro hacer como ...",@MartinOrozcoAgs Felicitaciones a Usted y a Gr...
1,@MarinadelPilar https://t.co/6mKV4rw4Wh,"@MartinOrozcoAgs Rogelio Guerrero, de Aguascal..."
2,@MarinadelPilar Y lo que venga,@MartinOrozcoAgs @GobAgs @LeoMontanezC @proesp...
3,@MarinadelPilar Y en delincuencia/crimen organ...,"@MartinOrozcoAgs estoy desesperada, por favor!!"
4,@MarinadelPilar “No les vamos a fallar” dijo l...,"@MartinOrozcoAgs por favor, necesito platicar ..."
...,...,...
37957,@LorenaCuellar @CulturaTlaxcala Lic Lorena Cue...,@DavidMonrealA Jajajaja viejo Ridiculo
37958,"@LorenaCuellar Y seguiremos avanzando, con tu ...",@DavidMonrealA Y en temas de seguridad para cu...
37959,@LorenaCuellar @FomtlaxOficial Pero para los t...,"@DavidMonrealA Pues que oportuno, una semana a..."
37960,@LorenaCuellar ¿Que paso? Dicen la líderes sin...,@DavidMonrealA Enhorabuena 👏


In [16]:
beto = 'dccuchile/bert-base-spanish-wwm-cased'

In [17]:
AL = "CenIA/albert-tiny-spanish"

Convertir de DataFrame a objeto Dataset para Hugginface

In [18]:
ds = Dataset.from_pandas(raw_dataset,preserve_index=True)

In [19]:
ds = ds.rename_column('__index_level_0__','id')

In [20]:
ds

Dataset({
    features: ['mujeres_tweet', 'hombres_tweet', 'id'],
    num_rows: 37962
})

In [21]:
ds = ds.train_test_split(shuffle = True, seed = 200, test_size=0.05)

In [22]:
ds

DatasetDict({
    train: Dataset({
        features: ['mujeres_tweet', 'hombres_tweet', 'id'],
        num_rows: 36063
    })
    test: Dataset({
        features: ['mujeres_tweet', 'hombres_tweet', 'id'],
        num_rows: 1899
    })
})

In [23]:
ds["train"][0]

{'mujeres_tweet': '@Claudiashein @MetroCDMX @CFEmx Bravo doctora!! Jamas se vieron tantos avances en la ciudad.',
 'hombres_tweet': '@CuitlahuacGJ Señor Gobernador me dirijo usted de manera respetuosa para pedirle me conceda una audiencia pues me ha costado un poco acceder a usted, se trata de un caso de ABUSO DE AUTORIDAD donde mi hija fue víctima, somos de el municipio de ALVARADO… Le pido me ayude a hacer JUSTICIA',
 'id': 35343}

Split datasets - 20% para test.

## Tokenization

In [24]:
tokz = BertTokenizer.from_pretrained(beto, do_lower_case=False)

In [19]:
tokz2 = AlbertTokenizer.from_pretrained(AL, do_lower_case=False)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [25]:
tokz.tokenize("hola mundo; soy Pao.")

['hola', 'mundo', ';', 'soy', 'Pa', '##o', '.']

In [26]:
ds["train"].features

{'mujeres_tweet': Value(dtype='string', id=None),
 'hombres_tweet': Value(dtype='string', id=None),
 'id': Value(dtype='int64', id=None)}

In [28]:
max_input_length = 280
max_target_length = 280
source_lang = "es"
target_lang = "es"
prefix = "translate spanish_to_women to spanish_to_men: "

def preprocess_function(examples):
    inputs = [ex for ex in examples["mujeres_tweet"]]
    targets = [ex for ex in examples["hombres_tweet"]]

    model_inputs = tokz(inputs,
                        text_pair=targets,
                        padding="max_length", 
                        truncation=True,
                        max_length=max_input_length,
                        #return_tensors = 'pt'
                        )

    #model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
ds["train"]["mujeres_tweet"][:5]

['@Claudiashein Muy bieen',
 '@kenialopezr Wooooo si izo su tarea conocer toda la cdmx q inteligente salió da pena',
 '@rocionahle Buena respuesta al tomandante borolas.',
 '@kenialopezr Todo el país debería tener la seguridad y tranquilidad que se dan los gobiernos panistas, por ejemplo Guanajuato y Tamaulipas!! 😂 Son mediocre, mentirosos, hipócritas y muy malos para gobernar, creen que ya olvidamos lo que pasó con el sin sesos de Fox y el comandante borolas?  https://t.co/9ZHqW4CDBQ',
 '@kenialopezr Vas y chingas tu puta madre,cara de caballo,si vienes por mi alcaldía,probaras un poco del amor del pueblo.']

In [23]:
res = not all(ds["train"]["hombres_tweet"])
 
# printing result
print("Does Dictionary contain None value? " + str(res))

Does Dictionary contain None value? False


In [29]:
en_sentence = ds["train"][100]["mujeres_tweet"]
fr_sentence = ds["train"][100]["hombres_tweet"]

inputs = tokz(en_sentence, text_pair=fr_sentence)
inputs

{'input_ids': [4, 968, 24179, 7642, 27721, 968, 1114, 9441, 18403, 6745, 948, 11177, 30935, 1017, 1512, 1759, 2521, 1009, 1045, 4640, 5269, 1181, 972, 972, 1031, 1009, 1345, 972, 982, 30967, 30991, 992, 30946, 30948, 30963, 30952, 30974, 30973, 5, 968, 19881, 1623, 948, 8418, 1340, 11801, 1036, 2116, 1008, 4272, 1110, 1110, 1323, 1793, 993, 4492, 11690, 1051, 9967, 4545, 1074, 1008, 5036, 1244, 2529, 1110, 26040, 1096, 1673, 3, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [30]:
tokenized_dataset = ds.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f90232d04c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [31]:
tokenized_dataset["train"]

Dataset({
    features: ['mujeres_tweet', 'hombres_tweet', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36063
})

In [32]:
gc.collect()

49

## Transformer (fingers crossed)

In [36]:
from transformers import AutoConfig, AutoModel
from transformers import TrainingArguments,Trainer, DataCollatorForLanguageModeling
from transformers import AutoModelForMaskedLM
from transformers import AlbertForPreTraining
from transformers import AlbertConfig
from transformers import BertForMaskedLM

Descargar la configuracion de BETO

In [34]:
gc.collect()

22

In [35]:
config = AutoConfig.from_pretrained(beto,low_cpu_mem_usage=True)

In [37]:
model2 = BertForMaskedLM.from_pretrained(beto)

In [38]:
d_collator = DataCollatorForLanguageModeling(tokenizer=tokz,return_tensors='pt')

## Training preliminar con Optuna

In [39]:
#import optuna
from transformers import create_optimizer, AdamW, Adafactor

In [54]:
from accelerate import Accelerator

In [177]:
#accelerator = Accelerator(mixed_precision='fp16')
#model, optimizer = accelerator.prepare(model2,Adafactor)

In [40]:
gc.collect()

430

In [41]:
import torch
torch.cuda.empty_cache()

In [42]:
#Curso de Jeremy Howard
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [58]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model2.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [59]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    optim='adafactor',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=2,
    fp16=True,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [60]:
trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokz,
    data_collator=d_collator,
    compute_metrics=corr_d
)

Using cuda_amp half precision backend


In [61]:
torch.cuda.empty_cache() 

In [62]:
gc.collect()

2050

In [63]:
model2.zero_grad()

In [64]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: id, mujeres_tweet, hombres_tweet. If id, mujeres_tweet, hombres_tweet are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 36063
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 562


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.69 GiB total capacity; 2.24 GiB already allocated; 11.56 MiB free; 2.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF